In [21]:
import pandas as pd
from workalendar.europe import Turkey
import holidays

# MAIN CODE
####################################################################
############## CREATE RELIGIOUS DATAFRAME ##########################
####################################################################

# Takvim objesi oluştur
cal = Turkey()

dates = []
names = []

for holiday in holidays.Turkey(years=range(2022, 2032)).items():
    date = holiday[0].strftime('%Y-%m-%d')
    name = holiday[1]
    if "Ramadan Feast"  in name  or  "Sacrifice Feast" in name:
        names.append(name)

        dates.append(date)
    

# Special days adında bir sözlük oluştur
special_days = {}

# Tarihleri anahtar ve özel gün adlarını değer olarak alarak sözlük oluştur
for date, name in zip(dates, names):
    if date not in special_days:
        special_days[date] = [name]
    else:
        special_days[date].append(name)

# Veri çerçevesini oluştur
df = pd.DataFrame.from_dict(special_days, orient='index', columns=['name'])

# İndex'i tarih sütununa ayarla
df.index.name = 'date'
df_religious = df.reset_index()

df_religious = df_religious.sort_values('date')
df_religious = pd.get_dummies(df_religious, columns=['name'], prefix='', prefix_sep='')
df_religious.reset_index(drop=True, inplace=True)



# Create a list of all dates between 2022-01-01 and 2031-12-31
date_range = pd.date_range(start='2022-01-01', end='2031-12-31', freq='D').strftime('%Y-%m-%d')

# Create a DataFrame with all dates in the range
df_all_dates = pd.DataFrame({'date': date_range})

# Merge the special days DataFrame with the all dates DataFrame
df_religious_final = pd.merge(df_all_dates, df_religious, on='date', how='left')

# Fill NaN values with 0
df_religious_final.fillna(0, inplace=True)
#one hot encoding
# convert float to int
convert_float_to_int = [i for i in df_religious_final if i != "date"]
df_religious_final[convert_float_to_int] = df_religious_final[convert_float_to_int].astype(int)

print(df_religious_final.head())
print(df_religious_final.columns)

renamed_col_ramadan = [col for col in df_religious_final.columns if 'Ramadan Feast' in col]#list comprehension
# "Ramadan Feast" ile başlayan sütunları topla ve yeni sütunu oluştur
df_religious_final['Ramadan Feast'] = df_religious_final[renamed_col_ramadan].sum(axis=1)
# İlgili sütunları listeden kaldır
df_religious_final = df_religious_final.drop(renamed_col_ramadan, axis=1)

renamed_col_sacrifice_feast = [col for col in df_religious_final.columns if 'Sacrifice Feast' in col]
# "Ramadan Feast" ile başlayan sütunları topla ve yeni sütunu oluştur
df_religious_final['Sacrifice Feast'] = df_religious_final[renamed_col_sacrifice_feast].sum(axis=1)
# İlgili sütunları listeden kaldır
df_religious_final = df_religious_final.drop(renamed_col_sacrifice_feast, axis=1)

#####################################################
########################
##############

# Her yılın Ramazan Bayramı'nın ilk gününden sonraki iki gün için Ramadan Feast sütununa 1 etiketi yerleştir
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Ramadan Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)
    df_religious_final.loc[start_idx+1:start_idx+2, 'Ramadan Feast'] = 1
    
    
    
# Her yılın Kurban Bayramı'nın ilk gününden sonraki üç gün için Sacrifice Feast sütununa 1 etiketi yerleştir
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Sacrifice Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)
    df_religious_final.loc[start_idx+1:start_idx+3, 'Sacrifice Feast'] = 1
    
##############
#########################
######################################################
    
###############################################################################################################################################
## note: 
## Hicri takvim bir Ay takvimi olduğu için bir yılın uzunluğu, miladi takvimden 11-12 gün daha kısadır.
## Bunun sonucu olarak Ramazan ayı her sene miladi takvimde öne kayar. Yaklaşık olarak her 32 senede bir, Ramazan ayı aynı tarihlere denk gelir.
################################################################################################################################################

# Her yıl Kurban bayramıu öncesi 30 günü oruç şeklinde etiketleme
df_religious_final["Oruc"] = 0
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Ramadan Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)

    if str(year) == "2023":
        df_religious_final.loc[start_idx-29:start_idx-1, 'Oruc'] = 1
    else:
        df_religious_final.loc[start_idx-30:start_idx-1, 'Oruc'] = 1
        
df_religious_final['date'] =  pd.to_datetime(df_religious_final['date'])



####################################################################
############## CREATE NON-RELIGIOUS DATAFRAME ######################
####################################################################

# Tatil isimlerinin ve tarihlerinin bulunduğu sözlük
holidays = {
    'New Year\'s Day': '01-01',
    'National Sovereignty and Children\'s Day': '04-23',
    'Labour Day': '05-01',
    'Youth and Sports Day': '05-19',
    'Democracy and National Unity Day': '07-15',
    'Victory Day': '08-30',
    'Republic Day': '10-29'
}

# 2022-2031 yılları arasındaki tarihleri oluştur
dates = pd.date_range(start='2022-01-01', end='2031-12-31')

# Boş bir DataFrame oluştur
df_without_religious = pd.DataFrame(index=dates)

# Tatilleri DataFrame'e ekle
for holiday, date in holidays.items():
    df_without_religious[holiday] = (df_without_religious.index.strftime('%m-%d') == date).astype(int)

# One-hot encode yap
df_without_religious_encoded = pd.get_dummies(df_without_religious, prefix='', prefix_sep='')


df_without_religious_encoded = df_without_religious_encoded.reset_index()

df_without_religious_encoded = df_without_religious_encoded.rename(columns={"index":"date"})


# merge religious and non-religious holidays
df_holidays_final = df_without_religious_encoded.merge(df_religious_final)
df_holidays_final = df_holidays_final.rename(columns={"date":"ds"})
df_holidays_final



         date  \
0  2022-01-01   
1  2022-01-02   
2  2022-01-03   
3  2022-01-04   
4  2022-01-05   

   Commemoration of Ataturk, Youth and Sports Day; Sacrifice Feast Holiday* (*estimated)  \
0                                                  0                                       
1                                                  0                                       
2                                                  0                                       
3                                                  0                                       
4                                                  0                                       

   National Sovereignty and Children's Day; Ramadan Feast Holiday* (*estimated)  \
0                                                  0                              
1                                                  0                              
2                                                  0                              
3           

,ds,New Year's Day,National Sovereignty and Children's Day,Labour Day,Youth and Sports Day,Democracy and National Unity Day,Victory Day,Republic Day,Ramadan Feast,Sacrifice Feast,Oruc
0,2022-01-01,1,0,0,0,0,0,0,0,0,0
1,2022-01-02,0,0,0,0,0,0,0,0,0,0
2,2022-01-03,0,0,0,0,0,0,0,0,0,0
3,2022-01-04,0,0,0,0,0,0,0,0,0,0
4,2022-01-05,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3647,2031-12-27,0,0,0,0,0,0,0,0,0,0
3648,2031-12-28,0,0,0,0,0,0,0,0,0,0
3649,2031-12-29,0,0,0,0,0,0,0,0,0,0
3650,2031-12-30,0,0,0,0,0,0,0,0,0,0


In [27]:
df_religious_final[renamed_col_ramadan]

KeyError: "None of [Index(['National Sovereignty and Children's Day; Ramadan Feast Holiday* (*estimated)',\n       'Ramadan Feast Holiday* (*estimated)', 'Ramadan Feast* (*estimated)'],\n      dtype='object')] are in the [columns]"

In [32]:
df_religious_final[["Ramadan Feast","Sacrifice Feast"]]

,Ramadan Feast,Sacrifice Feast
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
3647,0,0
3648,0,0
3649,0,0
3650,0,0


In [29]:
renamed_col_ramadan

["National Sovereignty and Children's Day; Ramadan Feast Holiday* (*estimated)",
 'Ramadan Feast Holiday* (*estimated)',
 'Ramadan Feast* (*estimated)']

In [17]:
df_holidays_final.to_csv("Holidays.csv", index=False)

In [18]:
pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
from workalendar.europe import Turkey
import holidays


####################################################################
############## CREATE RELIGIOUS DATAFRAME ##########################
####################################################################

# Takvim objesi oluştur
cal = Turkey()

dates = []
names = []

for holiday in holidays.Turkey(years=range(2022, 2032)).items():
    date = holiday[0].strftime('%Y-%m-%d')
    name = holiday[1]
    if name  == "Ramadan Feast"  or  name == "Sacrifice Feast":
        names.append(name)

        dates.append(date)
    

# Special days adında bir sözlük oluştur
special_days = {}

# Tarihleri anahtar ve özel gün adlarını değer olarak alarak sözlük oluştur
for date, name in zip(dates, names):
    if date not in special_days:
        special_days[date] = [name]
    else:
        special_days[date].append(name)

# Veri çerçevesini oluştur
df = pd.DataFrame.from_dict(special_days, orient='index', columns=['name'])

# İndex'i tarih sütununa ayarla
df.index.name = 'date'
df_religious = df.reset_index()

df_religious = df_religious.sort_values('date')
df_religious = pd.get_dummies(df_religious, columns=['name'], prefix='', prefix_sep='')
df_religious.reset_index(drop=True, inplace=True)



# Create a list of all dates between 2022-01-01 and 2031-12-31
date_range = pd.date_range(start='2022-01-01', end='2031-12-31', freq='D').strftime('%Y-%m-%d')

# Create a DataFrame with all dates in the range
df_all_dates = pd.DataFrame({'date': date_range})

# Merge the special days DataFrame with the all dates DataFrame
df_religious_final = pd.merge(df_all_dates, df_religious, on='date', how='left')

# Fill NaN values with 0
df_religious_final.fillna(0, inplace=True)

# convert float to int
convert_float_to_int = [i for i in df_religious_final if i != "date"]
df_religious_final[convert_float_to_int] = df_religious_final[convert_float_to_int].astype(int)




# Her yılın Ramazan Bayramı'nın ilk gününden sonraki iki gün için Ramadan Feast sütununa 1 etiketi yerleştir
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Ramadan Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)
    df_religious_final.loc[start_idx+1:start_idx+2, 'Ramadan Feast'] = 1
    
    
    
# Her yılın Kurban Bayramı'nın ilk gününden sonraki üç gün için Sacrifice Feast sütununa 1 etiketi yerleştir
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Sacrifice Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)
    df_religious_final.loc[start_idx+1:start_idx+3, 'Sacrifice Feast'] = 1
    

###############################################################################################################################################
## note: 
## Hicri takvim bir Ay takvimi olduğu için bir yılın uzunluğu, miladi takvimden 11-12 gün daha kısadır.
## Bunun sonucu olarak Ramazan ayı her sene miladi takvimde öne kayar. Yaklaşık olarak her 32 senede bir, Ramazan ayı aynı tarihlere denk gelir.
################################################################################################################################################

# Her yıl Kurban bayramıu öncesi 30 günü oruç şeklinde etiketleme
df_religious_final["Oruc"] = 0
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Ramadan Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)

    if str(year) == "2023":
        df_religious_final.loc[start_idx-29:start_idx-1, 'Oruc'] = 1
    else:
        df_religious_final.loc[start_idx-30:start_idx-1, 'Oruc'] = 1
        
df_religious_final['date'] =  pd.to_datetime(df_religious_final['date'])



####################################################################
############## CREATE NON-RELIGIOUS DATAFRAME ######################
####################################################################

# Tatil isimlerinin ve tarihlerinin bulunduğu sözlük
holidays = {
    'New Year\'s Day': '01-01',
    'National Sovereignty and Children\'s Day': '04-23',
    'Labour Day': '05-01',
    'Youth and Sports Day': '05-19',
    'Democracy and National Unity Day': '07-15',
    'Victory Day': '08-30',
    'Republic Day': '10-29'
}

# 2022-2031 yılları arasındaki tarihleri oluştur
dates = pd.date_range(start='2022-01-01', end='2031-12-31')

# Boş bir DataFrame oluştur
df_without_religious = pd.DataFrame(index=dates)

# Tatilleri DataFrame'e ekle
for holiday, date in holidays.items():
    df_without_religious[holiday] = (df_without_religious.index.strftime('%m-%d') == date).astype(int)

# One-hot encode yap
df_without_religious_encoded = pd.get_dummies(df_without_religious, prefix='', prefix_sep='')


df_without_religious_encoded = df_without_religious_encoded.reset_index()

df_without_religious_encoded = df_without_religious_encoded.rename(columns={"index":"date"})


# merge religious and non-religious holidays
df_holidays_final = df_without_religious_encoded.merge(df_religious_final)
df_holidays_final = df_holidays_final.rename(columns={"date":"ds"})
df_holidays_final

KeyError: 'Ramadan Feast'

In [3]:
# Takvim objesi oluştur
cal = Turkey()

dates = []
names = []

for holiday in holidays.Turkey(years=range(2022, 2032)).items():
    date = holiday[0].strftime('%Y-%m-%d')
    name = holiday[1]
    if name  == "Ramadan Feast"  or  name == "Sacrifice Feast":
        names.append(name)

        dates.append(date)

In [4]:
names

[]

In [20]:
for holiday in holidays.Turkey(years=range(2022, 2032)).items():
    date = holiday[0].strftime('%Y-%m-%d')
    name = holiday[1]
    print(f"Date: {date}, Name: {name}")
    if "Ramadan Feast" in name or "Sacrifice Feast" in name:
        names.append(name)
        dates.append(date)


Date: 2022-01-01, Name: New Year's Day
Date: 2022-04-23, Name: National Sovereignty and Children's Day
Date: 2022-05-01, Name: Labour Day
Date: 2022-05-19, Name: Commemoration of Ataturk, Youth and Sports Day
Date: 2022-07-15, Name: Democracy and National Unity Day
Date: 2022-08-30, Name: Victory Day
Date: 2022-10-29, Name: Republic Day
Date: 2022-05-02, Name: Ramadan Feast* (*estimated)
Date: 2022-05-03, Name: Ramadan Feast Holiday* (*estimated)
Date: 2022-05-04, Name: Ramadan Feast Holiday* (*estimated)
Date: 2022-07-09, Name: Sacrifice Feast* (*estimated)
Date: 2022-07-10, Name: Sacrifice Feast Holiday* (*estimated)
Date: 2022-07-11, Name: Sacrifice Feast Holiday* (*estimated)
Date: 2022-07-12, Name: Sacrifice Feast Holiday* (*estimated)
Date: 2023-01-01, Name: New Year's Day
Date: 2023-04-23, Name: National Sovereignty and Children's Day; Ramadan Feast Holiday* (*estimated)
Date: 2023-05-01, Name: Labour Day
Date: 2023-05-19, Name: Commemoration of Ataturk, Youth and Sports Day
Dat

In [8]:
names

['Ramadan Feast* (*estimated)',
 'Ramadan Feast Holiday* (*estimated)',
 'Ramadan Feast Holiday* (*estimated)',
 'Sacrifice Feast* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 "National Sovereignty and Children's Day; Ramadan Feast Holiday* (*estimated)",
 'Ramadan Feast* (*estimated)',
 'Ramadan Feast Holiday* (*estimated)',
 'Sacrifice Feast* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Ramadan Feast* (*estimated)',
 'Ramadan Feast Holiday* (*estimated)',
 'Ramadan Feast Holiday* (*estimated)',
 'Sacrifice Feast* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Sacrifice Feast Holiday* (*estimated)',
 'Ramadan Feast* (*estimated)',
 'Ramadan Feast Holiday* (*estimated)',
 'Ramadan Feast Holiday* (*estimated)',
 'Sacrifice Feast* (*estimated)

In [17]:
import pandas as pd
from workalendar.europe import Turkey
import holidays


####################################################################
############## CREATE RELIGIOUS DATAFRAME ##########################
####################################################################

# Takvim objesi oluştur
cal = Turkey()

dates = []
names = []

for holiday in holidays.Turkey(years=range(2022, 2032)).items():
    date = holiday[0].strftime('%Y-%m-%d')
    name = holiday[1]
    if "Ramadan Feast" in name or "Sacrifice Feast" in name:
        names.append(name)

        dates.append(date)
        
        
print(names)
print(date)
    

# Special days adında bir sözlük oluştur
special_days = {}

# Tarihleri anahtar ve özel gün adlarını değer olarak alarak sözlük oluştur
for date, name in zip(dates, names):
    if date not in special_days:
        special_days[date] = [name]
    else:
        special_days[date].append(name)

# Veri çerçevesini oluştur
df = pd.DataFrame.from_dict(special_days, orient='index', columns=['name'])

# İndex'i tarih sütununa ayarla
df.index.name = 'date'
df_religious = df.reset_index()

df_religious = df_religious.sort_values('date')
df_religious = pd.get_dummies(df_religious, columns=['name'], prefix='', prefix_sep='')
df_religious.reset_index(drop=True, inplace=True)



# Create a list of all dates between 2022-01-01 and 2031-12-31
date_range = pd.date_range(start='2022-01-01', end='2031-12-31', freq='D').strftime('%Y-%m-%d')

# Create a DataFrame with all dates in the range
df_all_dates = pd.DataFrame({'date': date_range})

# Merge the special days DataFrame with the all dates DataFrame
df_religious_final = pd.merge(df_all_dates, df_religious, on='date', how='left')

# Fill NaN values with 0
df_religious_final.fillna(0, inplace=True)

# convert float to int
convert_float_to_int = [i for i in df_religious_final if i != "date"]
df_religious_final[convert_float_to_int] = df_religious_final[convert_float_to_int].astype(int)

print([col for col in df_religious_final.columns if "Ramadan Feast" in col or "Sacrifice Feast" in col])

# Her yılın Ramazan Bayramı'nın ilk gününden sonraki iki gün için Ramadan Feast sütununa 1 etiketi yerleştir
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Ramadan Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)
    df_religious_final.loc[start_idx+1:start_idx+2, 'Ramadan Feast'] = 1
    
    
    
# Her yılın Kurban Bayramı'nın ilk gününden sonraki üç gün için Sacrifice Feast sütununa 1 etiketi yerleştir
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Sacrifice Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)
    df_religious_final.loc[start_idx+1:start_idx+3, 'Sacrifice Feast'] = 1
    

###############################################################################################################################################
## note: 
## Hicri takvim bir Ay takvimi olduğu için bir yılın uzunluğu, miladi takvimden 11-12 gün daha kısadır.
## Bunun sonucu olarak Ramazan ayı her sene miladi takvimde öne kayar. Yaklaşık olarak her 32 senede bir, Ramazan ayı aynı tarihlere denk gelir.
################################################################################################################################################

# Her yıl Kurban bayramıu öncesi 30 günü oruç şeklinde etiketleme
df_religious_final["Oruc"] = 0
for year in range(2022, 2032):
    start_date = df_religious_final[(df_religious_final['date'].str.contains(str(year)) ) & (df_religious_final['Ramadan Feast'] == 1)].index[0]
    start_idx = df_religious_final.index.get_loc(start_date)

    if str(year) == "2023":
        df_religious_final.loc[start_idx-29:start_idx-1, 'Oruc'] = 1
    else:
        df_religious_final.loc[start_idx-30:start_idx-1, 'Oruc'] = 1
        
df_religious_final['date'] =  pd.to_datetime(df_religious_final['date'])



####################################################################
############## CREATE NON-RELIGIOUS DATAFRAME ######################
####################################################################

# Tatil isimlerinin ve tarihlerinin bulunduğu sözlük
holidays = {
    'New Year\'s Day': '01-01',
    'National Sovereignty and Children\'s Day': '04-23',
    'Labour Day': '05-01',
    'Youth and Sports Day': '05-19',
    'Democracy and National Unity Day': '07-15',
    'Victory Day': '08-30',
    'Republic Day': '10-29'
}

# 2022-2031 yılları arasındaki tarihleri oluştur
dates = pd.date_range(start='2022-01-01', end='2031-12-31')

# Boş bir DataFrame oluştur
df_without_religious = pd.DataFrame(index=dates)

# Tatilleri DataFrame'e ekle
for holiday, date in holidays.items():
    df_without_religious[holiday] = (df_without_religious.index.strftime('%m-%d') == date).astype(int)

# One-hot encode yap
df_without_religious_encoded = pd.get_dummies(df_without_religious, prefix='', prefix_sep='')


df_without_religious_encoded = df_without_religious_encoded.reset_index()

df_without_religious_encoded = df_without_religious_encoded.rename(columns={"index":"date"})


# merge religious and non-religious holidays
df_holidays_final = df_without_religious_encoded.merge(df_religious_final)
df_holidays_final = df_holidays_final.rename(columns={"date":"ds"})
df_holidays_final

['Ramadan Feast* (*estimated)', 'Ramadan Feast Holiday* (*estimated)', 'Ramadan Feast Holiday* (*estimated)', 'Sacrifice Feast* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', "National Sovereignty and Children's Day; Ramadan Feast Holiday* (*estimated)", 'Ramadan Feast* (*estimated)', 'Ramadan Feast Holiday* (*estimated)', 'Sacrifice Feast* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Ramadan Feast* (*estimated)', 'Ramadan Feast Holiday* (*estimated)', 'Ramadan Feast Holiday* (*estimated)', 'Sacrifice Feast* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Sacrifice Feast Holiday* (*estimated)', 'Ramadan Feast* (*estimated)', 'Ramadan Feast Holiday* (*estimated)', 'Ramadan Feast Holiday* (*estimated)', 'Sacrifice Feast* (*estimated)', 'Sacrifice Feast Holi

KeyError: 'Ramadan Feast'